In [1]:
pip install tabulate

Note: you may need to restart the kernel to use updated packages.


# Predicting Power Outages and Their Cause

**Name(s)**: Ava Jeong and Charlene Hsu

**Website Link**: https://github.com/charl3n3hsu/DSC80_Final_Proj

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib_inline.backend_inline import set_matplotlib_formats
from IPython.display import display, IFrame, HTML

import plotly
import plotly.figure_factory as ff
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.io as pio

In [3]:
import pandas as pd
import numpy as np
from pathlib import Path

import plotly.express as px
pd.options.plotting.backend = 'plotly'

from dsc80_utils import *

## Step 1: Introduction

In [4]:
# TODO

## Step 2: Data Cleaning and Exploratory Data Analysis

In [5]:
outages = pd.read_csv('outages.csv')
outages = outages.iloc[1:]
outages

,variables,OBS,YEAR,MONTH,...,AREAPCT_UC,PCT_LAND,PCT_WATER_TOT,PCT_WATER_INLAND
1,NaN,1.0,2011.0,7.0,...,0.6,91.59266587,8.407334131,5.478742983
2,NaN,2.0,2014.0,5.0,...,0.6,91.59266587,8.407334131,5.478742983
3,NaN,3.0,2010.0,10.0,...,0.6,91.59266587,8.407334131,5.478742983
...,...,...,...,...,...,...,...,...,...
1532,NaN,1532.0,2009.0,8.0,...,0.15,98.30774418,1.692255822,1.692255822
1533,NaN,1533.0,2009.0,8.0,...,0.15,98.30774418,1.692255822,1.692255822
1534,NaN,1534.0,2000.0,NaN,...,0.02,85.76115446,14.23884554,2.901181874


In [6]:
outages.columns

Index(['variables', 'OBS', 'YEAR', 'MONTH', 'U.S._STATE', 'POSTAL.CODE',
       'NERC.REGION', 'CLIMATE.REGION', 'ANOMALY.LEVEL', 'CLIMATE.CATEGORY',
       'OUTAGE.START.DATE', 'OUTAGE.START.TIME', 'OUTAGE.RESTORATION.DATE',
       'OUTAGE.RESTORATION.TIME', 'CAUSE.CATEGORY', 'CAUSE.CATEGORY.DETAIL',
       'HURRICANE.NAMES', 'OUTAGE.DURATION', 'DEMAND.LOSS.MW',
       'CUSTOMERS.AFFECTED', 'RES.PRICE', 'COM.PRICE', 'IND.PRICE',
       'TOTAL.PRICE', 'RES.SALES', 'COM.SALES', 'IND.SALES', 'TOTAL.SALES',
       'RES.PERCEN', 'COM.PERCEN', 'IND.PERCEN', 'RES.CUSTOMERS',
       'COM.CUSTOMERS', 'IND.CUSTOMERS', 'TOTAL.CUSTOMERS', 'RES.CUST.PCT',
       'COM.CUST.PCT', 'IND.CUST.PCT', 'PC.REALGSP.STATE', 'PC.REALGSP.USA',
       'PC.REALGSP.REL', 'PC.REALGSP.CHANGE', 'UTIL.REALGSP', 'TOTAL.REALGSP',
       'UTIL.CONTRI', 'PI.UTIL.OFUSA', 'POPULATION', 'POPPCT_URBAN',
       'POPPCT_UC', 'POPDEN_URBAN', 'POPDEN_UC', 'POPDEN_RURAL',
       'AREAPCT_URBAN', 'AREAPCT_UC', 'PCT_LAND', 'PCT_WAT

In [7]:
# Keeping relevant columns
outages = outages[['YEAR', 'MONTH', 'U.S._STATE',
       'CLIMATE.REGION', 'ANOMALY.LEVEL', 'CLIMATE.CATEGORY',
       'OUTAGE.START.DATE', 'OUTAGE.START.TIME', 'OUTAGE.RESTORATION.DATE',
       'OUTAGE.RESTORATION.TIME', 'CAUSE.CATEGORY', 'CAUSE.CATEGORY.DETAIL',
       'HURRICANE.NAMES', 'OUTAGE.DURATION', 'DEMAND.LOSS.MW', 'CUSTOMERS.AFFECTED', 'POPULATION', 'POPPCT_URBAN',
       'POPDEN_URBAN', 'POPDEN_UC', 'POPDEN_RURAL',
       'AREAPCT_URBAN', 'AREAPCT_UC', 'PCT_LAND', 'PCT_WATER_TOT',
       'PCT_WATER_INLAND']]

In [8]:
print(outages.dtypes)

missing_values = outages[['MONTH', 'CLIMATE.REGION', 'CAUSE.CATEGORY', 'OUTAGE.DURATION']].isnull().sum()
print("Missing Values:\n", missing_values)

YEAR                float64
MONTH               float64
U.S._STATE           object
                     ...   
PCT_LAND             object
PCT_WATER_TOT        object
PCT_WATER_INLAND     object
Length: 26, dtype: object
Missing Values:
 MONTH               9
CLIMATE.REGION      6
CAUSE.CATEGORY      0
OUTAGE.DURATION    58
dtype: int64


In [9]:
outages['OUTAGE.DURATION'] = pd.to_numeric(outages['OUTAGE.DURATION'], errors='coerce')
outages['OUTAGE.DURATION'] = outages.groupby('CAUSE.CATEGORY')['OUTAGE.DURATION'].transform(lambda x: x.fillna(x.median()))
outages

,YEAR,MONTH,U.S._STATE,CLIMATE.REGION,...,AREAPCT_UC,PCT_LAND,PCT_WATER_TOT,PCT_WATER_INLAND
1,2011.0,7.0,Minnesota,East North Central,...,0.6,91.59266587,8.407334131,5.478742983
2,2014.0,5.0,Minnesota,East North Central,...,0.6,91.59266587,8.407334131,5.478742983
3,2010.0,10.0,Minnesota,East North Central,...,0.6,91.59266587,8.407334131,5.478742983
...,...,...,...,...,...,...,...,...,...
1532,2009.0,8.0,South Dakota,West North Central,...,0.15,98.30774418,1.692255822,1.692255822
1533,2009.0,8.0,South Dakota,West North Central,...,0.15,98.30774418,1.692255822,1.692255822
1534,2000.0,NaN,Alaska,NaN,...,0.02,85.76115446,14.23884554,2.901181874


In [10]:
# Imputation
category_mapping = outages.dropna(subset=['CAUSE.CATEGORY']).groupby('CAUSE.CATEGORY.DETAIL')['CAUSE.CATEGORY'].agg(pd.Series.mode)

def infer_category(detail):
    if pd.notna(detail) and detail in category_mapping:
        return category_mapping[detail]
    return None


outages['CAUSE.CATEGORY'] = outages.apply(lambda row: infer_category(row['CAUSE.CATEGORY.DETAIL']) if pd.isna(row['CAUSE.CATEGORY']) else row['CAUSE.CATEGORY'], axis=1)

outages['CAUSE.CATEGORY'] = outages['CAUSE.CATEGORY'].fillna(outages['CAUSE.CATEGORY'].mode()[0])

from sklearn.impute import SimpleImputer
def impute_missing_values(df):
    """
    Fill missing values in a DataFrame with appropriate imputation for each column type.

    Parameters:
    df (pd.DataFrame): The input DataFrame with missing values.

    Returns:
    pd.DataFrame: A DataFrame with missing values imputed.
    """
    df_imputed = df.copy()

    numerical_columns = df_imputed.select_dtypes(include=['number']).columns
    categorical_columns = df_imputed.select_dtypes(include=['object', 'category']).columns

    if len(numerical_columns) > 0:
        numerical_imputer = SimpleImputer(strategy='median')
        df_imputed[numerical_columns] = numerical_imputer.fit_transform(df_imputed[numerical_columns])

    if len(categorical_columns) > 0:
        categorical_imputer = SimpleImputer(strategy='most_frequent')
        df_imputed[categorical_columns] = categorical_imputer.fit_transform(df_imputed[categorical_columns])

    return df_imputed

impute_missing_values(outages)

outages['MONTH'] = outages['MONTH'].fillna(outages['MONTH'].median())

In [11]:
print(outages.isnull().sum())

YEAR                0
MONTH               0
U.S._STATE          0
                   ..
PCT_LAND            0
PCT_WATER_TOT       0
PCT_WATER_INLAND    0
Length: 26, dtype: int64


In [12]:
# HEAD of DF

outages.head()
# print(outages.head().to_markdown(index=False)) Include on WEBSITE

,YEAR,MONTH,U.S._STATE,CLIMATE.REGION,...,AREAPCT_UC,PCT_LAND,PCT_WATER_TOT,PCT_WATER_INLAND
1,2011.0,7.0,Minnesota,East North Central,...,0.6,91.59266587,8.407334131,5.478742983
2,2014.0,5.0,Minnesota,East North Central,...,0.6,91.59266587,8.407334131,5.478742983
3,2010.0,10.0,Minnesota,East North Central,...,0.6,91.59266587,8.407334131,5.478742983
4,2012.0,6.0,Minnesota,East North Central,...,0.6,91.59266587,8.407334131,5.478742983
5,2015.0,7.0,Minnesota,East North Central,...,0.6,91.59266587,8.407334131,5.478742983


In [13]:
# Frequency Distribution of Climate Region
fig1 = px.bar(outages['CLIMATE.REGION'].value_counts(), 
              title='Frequency Distribution of Climate Regions',
              labels={'index': 'Climate Region', 'value': 'Frequency'})
fig1.show()

In [14]:
# Boxplot of Outage Duration
fig2 = px.box(outages, y='OUTAGE.DURATION', title='Box Plot of Outage Duration',
              labels={'outage_duration': 'Outage Duration (hours)'})
fig2.show()

In [15]:
# Bivariate Analysis: Outage Duration vs. Climate Region
fig3 = px.box(outages, x='CLIMATE.REGION', y='OUTAGE.DURATION', 
              title='Outage Duration by Climate Region',
              labels={'climate_region': 'Climate Region', 'outage_duration': 'Outage Duration (hours)'})
fig3.show()


fig4 = px.strip(outages, x='CLIMATE.REGION', y='OUTAGE.DURATION', 
                title='Outage Duration by Climate Region (Scatter with Jitter)',
                labels={'climate_region': 'Climate Region', 'outage_duration': 'Outage Duration (hours)'})
fig4.show()

In [16]:
# Bivariate Analysis: Outage Duration vs. Cause Category
fig5 = px.box(outages, x='CAUSE.CATEGORY', y='OUTAGE.DURATION', 
              title='Outage Duration by Cause Category',
              labels={'cause_category': 'Cause Category', 'outage_duration': 'Outage Duration (hours)'})
fig5.show()


fig6 = px.strip(outages, x='CAUSE.CATEGORY', y='OUTAGE.DURATION', 
                title='Outage Duration by Cause Category (Scatter with Jitter)',
                labels={'cause_category': 'Cause Category', 'outage_duration': 'Outage Duration (hours)'})
fig6.show()

In [17]:
# Plotting

#Boxplot (BI-VARIATE)
fig = px.box(outages, x="CAUSE.CATEGORY", y="OUTAGE.DURATION", 
             title="Distribution of Outage Duration by Cause Category",
             points="all")  # Show all points

fig.show()

In [18]:
# Pivot Tables by Mean

pivot_table = pd.pivot_table(
    outages,
    values='OUTAGE.DURATION',  
    index='CLIMATE.REGION',   
    columns='CAUSE.CATEGORY', 
    aggfunc='mean',            
    fill_value=0              
)
pivot_table

CAUSE.CATEGORY,equipment failure,fuel supply emergency,intentional attack,islanding,public appeal,severe weather,system operability disruption
CLIMATE.REGION,,,,,,,
Central,293.14,10035.25,315.53,125.33,1410.00,3238.30,2469.73
East North Central,26435.33,27969.00,2376.05,1.00,733.00,4434.82,2610.00
Northeast,215.80,14629.57,191.84,881.00,2655.00,4418.71,736.27
...,...,...,...,...,...,...,...
Southwest,113.80,2018.00,255.84,2.00,2275.00,11572.90,329.22
West,524.81,5250.94,857.68,214.86,2028.11,2908.30,356.41
West North Central,61.00,3960.00,23.50,68.20,439.50,2442.50,0.00


In [19]:
# Bar (BI_VARIATE)
df_summary = outages.groupby("CAUSE.CATEGORY")["OUTAGE.DURATION"].agg(["mean", "median"]).reset_index()

fig = px.bar(df_summary, x="CAUSE.CATEGORY", y=["mean", "median"], 
             title="Mean and Median Outage Duration by Cause Category",
             barmode="group")

fig.update_layout(xaxis_title="Cause Category", yaxis_title="Duration (mins)")
fig.show()

## Step 3: Assessment of Missingness

In [20]:
# Permutation Test for Missingness

def permutation_test(data, col_missing, col_test, num_permutations=1000):
    cause_frequencies = data[col_test].value_counts(normalize=True).to_dict()
    
    data['CAUSE_ENCODED'] = data[col_test].map(cause_frequencies)
    
    observed_diff = abs(data[data[col_missing].isnull()]['CAUSE_ENCODED'].mean() -
                        data[~data[col_missing].isnull()]['CAUSE_ENCODED'].mean())
    
    perm_diffs = []
    for _ in range(num_permutations):
        shuffled = data['CAUSE_ENCODED'].sample(frac=1, replace=False).reset_index(drop=True)
        perm_diff = abs(data[data[col_missing].isnull()].index.to_series().map(shuffled).mean() -
                        data[~data[col_missing].isnull()].index.to_series().map(shuffled).mean())
        perm_diffs.append(perm_diff)
    
    p_value = np.mean(np.array(perm_diffs) >= observed_diff)
    
    fig = px.histogram(perm_diffs, nbins=30, title=f'Permutation Test for Missingness in {col_missing}', 
                       labels={'value': 'Test Statistic (Mean Difference)'}, opacity=0.7)
    fig.add_vline(x=observed_diff, line_dash='dash', line_color='red', annotation_text='Observed Difference')
    fig.show()
    
    return observed_diff, p_value

df = pd.read_csv('outages.csv').iloc[1:]
df['OUTAGE.DURATION'] = pd.to_numeric(df['OUTAGE.DURATION'], errors='coerce')
observed_diff, p_value = permutation_test(df, 'OUTAGE.DURATION', 'CAUSE.CATEGORY')
print(f"Observed Difference: {observed_diff}, P-Value: {p_value}")

Observed Difference: 0.08634898930475615, P-Value: 0.0


## Step 4: Hypothesis Testing

In [21]:
from scipy.stats import kruskal

# Drop rows with missing values in the columns of interest
outages_clean = outages.dropna(subset=['CAUSE.CATEGORY', 'OUTAGE.DURATION'])

# Group outage durations by cause category
groups = [group['OUTAGE.DURATION'].values for name, group in outages_clean.groupby('CAUSE.CATEGORY')]

# Perform Kruskal-Wallis test
statistic, p_value = kruskal(*groups)

print(f"Kruskal-Wallis Test Statistic: {statistic}, P-Value: {p_value}")

# Create a violin plot
fig = px.violin(outages_clean, x="CAUSE.CATEGORY", y="OUTAGE.DURATION", 
                title="Distribution of Outage Durations by Cause Category",
                box=True, points="all", hover_data=outages_clean.columns)

# Update layout
fig.update_layout(xaxis_title="Cause Category", yaxis_title="Outage Duration (mins)")
fig.show()

Kruskal-Wallis Test Statistic: 700.6054239816652, P-Value: 4.526908484457232e-148


In [22]:
# Permutation Test
def permutation_test_missingness(df, column_to_test, missingness_column, n_permutations=1000):
    """
    Perform a permutation test to check if the missingness of a column depends on another column.

    Parameters:
    df (pd.DataFrame): The input DataFrame.
    column_to_test (str): The column to test for dependency on missingness.
    missingness_column (str): The missingness indicator column.
    n_permutations (int): Number of permutations to perform.

    Returns:
    float: The p-value of the permutation test.
    """
    grouped = df.groupby(column_to_test)[missingness_column].mean()
    observed_tvd = np.abs(grouped.diff().dropna()).sum() / 2

    # Permutation test
    tvds = []
    for n in range(n_permutations):
        shuffled = df.assign(Shuffled=np.random.permutation(df[missingness_column]))

        shuffled_grouped = shuffled.groupby(column_to_test)['Shuffled'].mean()
        shuffled_tvd = np.abs(shuffled_grouped.diff().dropna()).sum() / 2
        tvds.append(shuffled_tvd)

    p_value = (np.array(tvds) >= observed_tvd).mean()

    return observed_tvd, tvds, p_value

In [23]:
# Perform the test for climate_region
observed_tvd, tvds, _ = permutation_test_missingness(outages, 'CLIMATE.REGION', 'OUTAGE.DURATION')

# Plot the empirical distribution of TVD
fig = px.histogram(tvds, nbins=30, labels={'value': 'TVD'}, title='Empirical Distribution of TVD under Null Hypothesis')
fig.add_vline(x=observed_tvd, line_dash="dash", line_color="red", annotation_text="Observed TVD", annotation_position="top right")
fig.update_layout(xaxis_title="TVD", yaxis_title="Frequency", showlegend=False)
fig.show()

In [24]:
# Perform the test for cause_category
observed_tvd, tvds, _ = permutation_test_missingness(outages, 'CAUSE.CATEGORY', 'OUTAGE.DURATION')

# Plot the empirical distribution of TVD
fig = px.histogram(tvds, nbins=30, labels={'value': 'TVD'}, title='Empirical Distribution of TVD under Null Hypothesis')
fig.add_vline(x=observed_tvd, line_dash="dash", line_color="red", annotation_text="Observed TVD", annotation_position="top right")
fig.update_layout(xaxis_title="TVD", yaxis_title="Frequency", showlegend=False)
fig.show()

## Step 5: Framing a Prediction Problem

In [25]:
# TODO

## Step 6: Baseline Model

In [26]:
# Linear Regression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

X = outages_clean[['CAUSE.CATEGORY', 'CLIMATE.REGION']] 
y = outages_clean['OUTAGE.DURATION'] 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

categorical_features = ['CAUSE.CATEGORY', 'CLIMATE.REGION']

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features) 
    ]
)

pipeline = Pipeline([
    ('preprocessor', preprocessor), 
    ('regressor', LinearRegression()) 
])

pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error (MSE): {mse}")
print(f"R² Score: {r2}")

Mean Squared Error (MSE): 26048694.311368402
R² Score: 0.007740974514328203


## Step 7: Final Model

In [27]:
# Feature Engineering
outages['DEMAND.LOSS.MW'] = outages['DEMAND.LOSS.MW'].astype(float)
outages['CUSTOMERS.AFFECTED'] = outages['CUSTOMERS.AFFECTED'].astype(float)

outages['DEMAND.LOSS.MW'] = outages['DEMAND.LOSS.MW'].fillna(outages['DEMAND.LOSS.MW'].median())
outages['CUSTOMERS.AFFECTED'] = outages['CUSTOMERS.AFFECTED'].fillna(outages['CUSTOMERS.AFFECTED'].median())


outages['DEMAND_PER_CUSTOMER'] = outages['DEMAND.LOSS.MW'] / outages['CUSTOMERS.AFFECTED']
outages['LOG_DEMAND_LOSS'] = np.log(outages['DEMAND.LOSS.MW'] + 1)

features = ['DEMAND.LOSS.MW', 'CUSTOMERS.AFFECTED', 'DEMAND_PER_CUSTOMER', 'LOG_DEMAND_LOSS', 'MONTH'] \
           + list(outages.filter(like='CAUSE.CATEGORY_').columns) \
           + list(outages.filter(like='CLIMATE.REGION_').columns)

X = outages[features]
y = outages['OUTAGE.DURATION']

In [32]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score

# Define features and target
features = ['DEMAND.LOSS.MW', 'CUSTOMERS.AFFECTED', 'DEMAND_PER_CUSTOMER', 'LOG_DEMAND_LOSS']
from sklearn.impute import SimpleImputer

# Impute missing values with the median
outages.replace([np.inf, -np.inf], np.nan, inplace=True)
X = pd.DataFrame(SimpleImputer(strategy='median').fit_transform(outages[features]), columns=features)
y = outages['OUTAGE.DURATION']

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a pipeline with PolynomialFeatures and LinearRegression
pipeline = Pipeline([
    ('poly', PolynomialFeatures()),  # Polynomial feature transformation
    ('linear', LinearRegression())   # Linear Regression model
])

# Define the parameter grid for PolynomialFeatures
param_grid = {
    'poly__degree': [1, 2, 3]  # Correct parameter name: 'poly__degree'
}

# Perform GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Get the best model
best_model = grid_search.best_estimator_

# Evaluate the best model
y_pred = best_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Best Degree: {grid_search.best_params_['poly__degree']}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"R² Score: {r2}")

Best Degree: 2
Mean Squared Error (MSE): 24668713.557935238
R² Score: 0.060307845668133964


## Step 8: Fairness Analysis

In [41]:
outages['CAUSE.CATEGORY'].value_counts()

CAUSE.CATEGORY
severe weather                   763
intentional attack               418
system operability disruption    127
public appeal                     69
equipment failure                 60
fuel supply emergency             51
islanding                         46
Name: count, dtype: int64

In [42]:
outages['natural_cause'] = outages['CAUSE.CATEGORY'].isin(['severe weather']).astype(int)

In [43]:
from sklearn.metrics import mean_squared_error

# Get predictions from the final model
y_pred = best_model.predict(X_test)

# Add predictions and true values to the test set
results = X_test.copy()
results['OUTAGE.DURATION'] = y_test
results['prediction'] = y_pred
results['OUTAGE.DURATION'] = results['OUTAGE.DURATION'].fillna(0)
results['prediction'] = results['prediction'].fillna(0)
results['natural_cause'] = outages.loc[results.index, 'natural_cause']  # Add group labels

# Observed RMSE for severe weather outages
rmse_severe_weather = np.sqrt(mean_squared_error(
    results[results['natural_cause'] == 1]['OUTAGE.DURATION'],
    results[results['natural_cause'] == 1]['prediction']
))

# Observed RMSE for other causes
rmse_other = np.sqrt(mean_squared_error(
    results[results['natural_cause'] == 0]['OUTAGE.DURATION'],
    results[results['natural_cause'] == 0]['prediction']
))

# Observed difference in RMSE
observed_diff = rmse_severe_weather - rmse_other
print(f"Observed Difference in RMSE (Severe Weather - Other Causes): {observed_diff}")

Observed Difference in RMSE (Severe Weather - Other Causes): 440.3600262801838


In [44]:
# Define a function to compute the difference in RMSE
def compute_rmse_diff(results):
    rmse_severe_weather = np.sqrt(mean_squared_error(
        results[results['natural_cause'] == 1]['OUTAGE.DURATION'],
        results[results['natural_cause'] == 1]['prediction']
    ))
    rmse_other = np.sqrt(mean_squared_error(
        results[results['natural_cause'] == 0]['OUTAGE.DURATION'],
        results[results['natural_cause'] == 0]['prediction']
    ))
    return rmse_severe_weather - rmse_other

# Permutation test
np.random.seed(42)  # For reproducibility
n_permutations = 500
diff_in_rmse = []

for _ in range(n_permutations):
    # Shuffle the 'natural_cause' column
    shuffled = results.assign(natural_cause=np.random.permutation(results['natural_cause']))
    
    # Compute the difference in RMSE for the shuffled data
    diff = compute_rmse_diff(shuffled)
    diff_in_rmse.append(diff)

# Compute p-value
p_value = (np.abs(diff_in_rmse) >= np.abs(observed_diff)).mean()
print(f"P-value: {p_value}")

P-value: 0.406


In [45]:
import plotly.express as px

# Create a histogram of permuted differences
fig = px.histogram(diff_in_rmse, nbins=20, labels={'value': 'Difference in RMSE'},
                   title='Permutation Test: Difference in RMSE (Severe Weather - Other Causes)')
fig.add_vline(x=observed_diff, line_color='red', annotation_text=f'Observed Difference: {observed_diff:.2f}')
fig.update_layout(xaxis_title='Difference in RMSE', yaxis_title='Frequency')
fig.show()